# FAIRHealth PHT PPDML

In [ ]:
import requests
import json
import uuid
import time
signalStation = "http://vwdata.personalhealthtrain.net"

def getStations(signalStation):
    response = requests.get(signalStation + "/client")
    return json.loads(response.text)

def defineTrain(image, inputVars, runId):
    return {
        "image": image,
        "inputString": ("%s" % json.dumps(inputVars)),
        "runId": runId
    }

def requestExecution(signalStation, clientId, taskDescription):
    response = requests.post(signalStation + "/client/" + str(clientId) + "/task/add", json=taskDescription)
    taskId = json.loads(response.text)["taskId"]
    return taskId

def getExecutionResult(signalStation, clientId, taskId, wait=True):
    url = signalStation + "/client/" + str(clientId) + "/task/" + str(taskId) + "/result"
    jsonResult = json.loads(requests.get(url).text)
    
    while len(jsonResult)==0 & wait:
        time.sleep(2)
        jsonResult = json.loads(requests.get(url).text)
    
    return jsonResult

## List Stations

In [ ]:
getStations(signalStation)

## Stage 1 at Maastricht Study

### Define train

In [ ]:
inputData = {
    "stage": 1
}
myTrain = defineTrain("fairhealth/ppdml", inputData, 1)

### Run on client

In [ ]:
clientId = 1
taskId = requestExecution(signalStation, clientId, myTrain)
print("Task submitted with ID: %s" % taskId)

### Receive result

In [ ]:
results = getExecutionResult(signalStation, clientId, taskId)
print(results[0]["log"])
result = json.loads(results[0]["response"])
print(result.keys())

## Stage 2 at CBS/Maastro endpoint

In [ ]:
result["stage"] = 2
mySecondTrain = defineTrain("fairhealth/ppdml", result, 1)

In [ ]:
clientId = 2
taskId = requestExecution(signalStation, clientId, mySecondTrain)
print("Task submitted with ID: %s" % taskId)

In [ ]:
resultsSecond = getExecutionResult(signalStation, clientId, taskId)
print(resultsSecond[0]["log"])
resultSecond = json.loads(resultsSecond[0]["response"])
print(resultSecond.keys())

sumNoisesAB = resultSecond["sumNoisesAB"]

## Stage 3 at DMS

In [ ]:
resultSecond["stage"] = 3
myThirdTrain = defineTrain("fairhealth/ppdml", resultSecond, 1)

In [ ]:
clientId = 1
taskId = requestExecution(signalStation, clientId, myThirdTrain)
print("Task submitted with ID: %s" % taskId)

In [ ]:
resultsThird = getExecutionResult(signalStation, clientId, taskId)
print(resultsThird[0]["log"])
resultThird = json.loads(resultsThird[0]["response"])
print(resultThird.keys())

## Stage 4 at CBS/Maastro

In [ ]:
resultThird["stage"] = 4
resultThird["sumNoisesAB"] = sumNoisesAB
myFourthTrain = defineTrain("fairhealth/ppdml", resultThird, 1)
print(resultThird.keys())

In [ ]:
clientId = 2
taskId = requestExecution(signalStation, clientId, myFourthTrain)
print("Task submitted with ID: %s" % taskId)

In [ ]:
resultsFourth = getExecutionResult(signalStation, clientId, taskId)
print(resultsFourth[0]["log"])
resultFourth = json.loads(resultsFourth[0]["response"])

In [ ]:
resultFourth